In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np, random

seed = 2025
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
train_transforms = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914,0.4822,0.4465), (0.2023,0.1994,0.2010))
])
test_transforms = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914,0.4822,0.4465), (0.2023,0.1994,0.2010))
])

train_data = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,            # ← change here
    transform=train_transforms
)
val_data = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,            # ← and here
    transform=test_transforms
)

train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=128, shuffle=True, num_workers=2, pin_memory=True
)
val_loader = torch.utils.data.DataLoader(
    val_data, batch_size=128, shuffle=False, num_workers=2, pin_memory=True
)

100%|██████████| 170M/170M [00:13<00:00, 12.7MB/s]


In [3]:
class ConvUnit(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
    def forward(self, x):
        return self.layer(x)

class CompactCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.blocks = nn.Sequential(
            ConvUnit(3,   64),   # 32→16
            ConvUnit(64, 128),   # 16→ 8
            ConvUnit(128,256)    #  8→ 4
        )
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier  = nn.Linear(256, 10)

    def forward(self, x):
        x = self.blocks(x)
        x = self.global_pool(x).view(x.size(0), -1)
        return self.classifier(x)

model = CompactCNN().to(device)

In [4]:
loss_fn      = nn.CrossEntropyLoss()
opt          = optim.AdamW(model.parameters(), lr=0.002, weight_decay=5e-4)
lr_schedule  = CosineAnnealingLR(opt, T_max=30)

epochs = 30
train_acc_list, val_acc_list = [], []

In [6]:
for ep in range(1, epochs+1):

    model.train()
    correct = total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        opt.zero_grad()
        outputs = model(inputs)
        loss    = loss_fn(outputs, labels)
        loss.backward()
        opt.step()

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)
    train_acc = 100 * correct / total
    train_acc_list.append(train_acc)

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            preds = model(inputs).argmax(dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)
    val_acc = 100 * correct / total
    val_acc_list.append(val_acc)

    lr_schedule.step()
    print(f"Epoch {ep:2d}/{epochs} | "
          f"Train {train_acc:5.2f}% | Test {val_acc:5.2f}%")

Epoch  1/30 | Train 83.41% | Test 77.41%
Epoch  2/30 | Train 84.90% | Test 82.18%
Epoch  3/30 | Train 86.74% | Test 82.81%
Epoch  4/30 | Train 87.73% | Test 84.57%
Epoch  5/30 | Train 88.98% | Test 84.35%
Epoch  6/30 | Train 89.68% | Test 85.45%
Epoch  7/30 | Train 90.65% | Test 86.18%
Epoch  8/30 | Train 91.57% | Test 88.13%
Epoch  9/30 | Train 92.28% | Test 87.36%
Epoch 10/30 | Train 92.98% | Test 84.80%
Epoch 11/30 | Train 93.79% | Test 88.78%
Epoch 12/30 | Train 94.22% | Test 88.97%
Epoch 13/30 | Train 94.88% | Test 88.27%
Epoch 14/30 | Train 95.49% | Test 89.03%
Epoch 15/30 | Train 96.11% | Test 89.57%
Epoch 16/30 | Train 96.26% | Test 89.52%
Epoch 17/30 | Train 97.02% | Test 90.05%
Epoch 18/30 | Train 97.49% | Test 90.10%
Epoch 19/30 | Train 97.56% | Test 90.24%
Epoch 20/30 | Train 97.96% | Test 90.57%
Epoch 21/30 | Train 98.16% | Test 90.67%
Epoch 22/30 | Train 98.42% | Test 90.58%
Epoch 23/30 | Train 98.53% | Test 90.80%
Epoch 24/30 | Train 98.65% | Test 90.84%
Epoch 25/30 | Tr